<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Simple Decision Prompts</span>
</div>

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundamental building blocks for eliciting effective responses from AI models. This module demonstrates how to use common prompt engineering techniques within Semantic Kernel. If you’ve used ChatGPT or Microsoft Copilot, you’re already familiar with prompting: given an instruction, a language model predicts the most likely and relevant response. With Semantic Kernel, you can build applications, services, and APIs that execute these prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal is to maximize the quality and clarity of the AI’s output, often by using specific wording, added context, or concrete examples within the prompt.

By combining Decision Intelligence with Prompt Engineering, you can create “Generative AI Decision Intelligence.” This approach leverages GenAI models to reason through complex tasks, gather intelligence, recommend decisions, and communicate results more effectively.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.5"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.5"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connection info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.5 Microsoft.Extensions.Configuration.Json, 9.0.5

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.54"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.54.0

Using Azure OpenAI Service


### Step 2 - Execute a Decision Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [3]:
// A Decision Intelligence prompt to help with describing decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
var response = simplePromptResponse.GetValue<string>(); 

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

Here are five decision-making frameworks that can enhance the quality of decisions, along with brief descriptions of how each supports better analysis and reasoning:

---

### 1. **The OODA Loop (Observe, Orient, Decide, Act)**  
**Overview:** Developed by military strategist John Boyd, this framework emphasizes rapid iteration through four stages:  
- **Observe:** Collect data and situational awareness.  
- **Orient:** Analyze information and align with context, goals, and mental models.  
- **Decide:** Choose the best course of action based on insights.  
- **Act:** Implement the decision while preparing for feedback.

**How it supports decision-making:**  
The OODA Loop facilitates agility in dynamic situations, helping individuals and organizations adapt to changing circumstances by constantly evaluating and refining their approach.

---

### 2. **Kepner-Tregoe Decision Analysis**  
**Overview:** This systematic, structured approach emphasizes the analysis of possible solutions based on objectives, problem identification, and evaluating risks. Key steps include:  
- Clarify the decision and objectives.  
- Identify and categorize alternatives.  
- Assess risks and mitigation strategies.  
- Choose the option that best meets criteria.

**How it supports decision-making:**  
By breaking decisions into objective components and focusing on logic, this method reduces bias, prioritizes goals, and carefully considers trade-offs, making it ideal for high-stakes challenges.

---

### 3. **Pros and Cons Matrix**  
**Overview:** This is a simple but effective method where positive outcomes (pros) and negative outcomes (cons) for each alternative are listed, weighed, and compared.

**How it supports decision-making:**  
The matrix encourages structured analysis and provides clarity, helping decision-makers visualize the advantages and disadvantages of each option to make more informed choices.

---

### 4. **Pareto Analysis (80/20 Rule)**  
**Overview:** This framework focuses on identifying the decisions or actions that will yield the most significant impact, based on the principle that 80% of results typically stem from 20% of causes.

**How it supports decision-making:**  
Pareto Analysis helps prioritize tasks or solutions by directing attention toward the most critical factors, thereby maximizing efficiency and impact while avoiding wasted effort.

---

### 5. **Six Thinking Hats (by Edward de Bono)**  
**Overview:** This method involves adopting six distinct perspectives sequentially to evaluate a problem or decision:  
- **White Hat:** Focus on facts and data.  
- **Red Hat:** Consider emotions and intuition.  
- **Black Hat:** Assess risks and potential downsides.  
- **Yellow Hat:** Highlight positive aspects.  
- **Green Hat:** Explore creativity and alternative solutions.  
- **Blue Hat:** Facilitate control and process strategy.

**How it supports decision-making:**  
By systematically evaluating decisions from multiple perspectives, this framework reduces bias, enhances creativity, and ensures comprehensive analysis.

---

These frameworks help decision-makers adopt structured, logical, and multi-dimensional approaches, enhancing both the quality and the reliability of their decisions in various scenarios.

### Step 3 - Execute a Decision Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

> 📝 Note: An average human can read between 25-40 Tokens / second. Therefore, while streaming certainly helps with providing AI output to the user, it begins to lose its effectiveness at large token velocity.  

In [4]:
// Decision Prompt using Streaming output chunks 
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

Here are five decision-making frameworks that can enhance the quality of decisions, along with brief explanations of how each supports better analysis and reasoning:

---

### 1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**  
**Description:** SWOT analysis is a strategic planning tool used to evaluate internal factors (strengths and weaknesses) and external factors (opportunities and threats).  

**How It Enhances Decision Making:**  
- **Holistic Evaluation:** Encourages a balanced view of internal capacities and external conditions.  
- **Priority Setting:** Helps identify areas for improvement and seize valuable opportunities.  
- **Clarity:** Provides structured insights, making complex situations easier to understand.  
- **Versatility:** Applicable to both individual and organizational decisions.  

---

### 2. **Pareto Analysis (80/20 Rule)**  
**Description:** Based on the principle that 80% of outcomes often come from 20% of causes, Pareto Analysis focuses

### Step 4 - Execute a Decision Prompt with Improved Output Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can output Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting output. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [7]:
// A new decision prompt to help with describing decision-making frameworks using Markdown format
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat);
var response = simplePromptResponse.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Decision-Making Framework       | Description                                                                                                      |
|---------------------------------|------------------------------------------------------------------------------------------------------------------|
| SWOT Analysis                   | Evaluates Strengths, Weaknesses, Opportunities, and Threats, providing a structured overview for strategic choices. |
| Decision Matrix                 | Scores and compares options against criteria to quantify preferences, enhancing objectivity in complex scenarios.  |
| Six Thinking Hats               | Encourages perspective-taking with distinct 'thinking hats' for emotions, logic, creativity, and more.             |
| Pareto Analysis                 | Prioritizes decisions based on the 80/20 rule, focusing on factors that yield the greatest impact.                 |
| Cost-Benefit Analysis           | Compares costs and benefits of options quantitatively, ensuring financial and resource efficiency.                 |

### Step 5 - Execute a Decision Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Temperature or other settings.

> 📝 Note: The supported prompt settings are dependent on the API plus the specific model version. For example, an AI model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available. Types of model execution (general versus reasoning) will also have different execution setting parameters. 

In [8]:
// Create a new OpenAI Prompt Execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Create a new KernelArguments object with the OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseWithOpenAI = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat, kernelArguments);
var response = promptResponseWithOpenAI.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Decision-Making Framework       | Description                                                                                     |
|---------------------------------|-------------------------------------------------------------------------------------------------|
| **SWOT Analysis**               | Evaluates Strengths, Weaknesses, Opportunities, and Threats to provide a structured overview of internal and external factors affecting a decision. |
| **Cost-Benefit Analysis**       | Compares the costs and benefits of a decision quantitatively or qualitatively to determine the most advantageous option. |
| **Pareto Analysis**             | Focuses on identifying the most significant factors (the 20% that cause 80% of the impact) to prioritize efforts effectively. |
| **Decision Matrix**             | Uses a grid to compare multiple options against weighted criteria, ensuring a systematic evaluation of alternatives. |
| **Six Thinking Hats**           | Encourages exploring decisions from six perspectives (e.g., logic, emotions, creativity) to ensure a balanced and thorough analysis. |

### Step 6 - Execute a Decision Prompt with a System Prompt (Custom AI Persona)

When building Decision Intelligence prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific (i.e. decision intelligence)
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something else
* Provide context (additional information) to the AI
* Using Roles in Chat Completion API prompts
* Give your AI words of encouragement  

A key best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a unique persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [10]:
// Create a System Prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);


var promptResponseWithTemplate = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt, kernelArguments);
var response = promptResponseWithTemplate.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

Here are five decision frameworks that are simple, versatile, and effective for daily situations. Each framework is easy to understand and can be applied across a range of scenarios, from personal decisions to professional problem-solving:

---

### **1. The Eisenhower Matrix (Urgent vs. Important)**
#### **Overview:**
This framework helps prioritize tasks and decisions based on urgency and importance. It’s particularly useful for time management and deciding what to focus on.

#### **How It Works:**
- Create a 2x2 matrix with "Urgent" and "Important" as axes.
- Categorize tasks or decisions into one of four quadrants:
  - **Quadrant 1:** Urgent & Important (Do immediately)
  - **Quadrant 2:** Not Urgent but Important (Schedule for later)
  - **Quadrant 3:** Urgent but Not Important (Delegate or minimize)
  - **Quadrant 4:** Neither Urgent nor Important (Eliminate or ignore)

#### **Example Use Case:**
Deciding how to spend your day when you have competing priorities like work deadlines, errands, and leisure activities.

---

### **2. The 10/10/10 Rule**
#### **Overview:**
This framework evaluates decisions based on their impact over three time horizons: 10 minutes, 10 months, and 10 years. It’s ideal for gaining perspective on short-term vs. long-term consequences.

#### **How It Works:**
- When faced with a decision, ask yourself:
  - How will I feel about this decision in **10 minutes**?
  - How will I feel about this decision in **10 months**?
  - How will I feel about this decision in **10 years**?
- Use the answers to weigh immediate gratification against long-term benefits.

#### **Example Use Case:**
Deciding whether to eat junk food or stick to a healthy diet.

---

### **3. The Pareto Principle (80/20 Rule)**
#### **Overview:**
This framework helps identify the most impactful actions or choices by focusing on the 20% of efforts that produce 80% of results. It’s great for optimizing decisions and avoiding wasted effort.

#### **How It Works:**
- List all possible options or tasks related to your decision.
- Identify the few key options that will deliver the majority of the desired outcomes.
- Focus your energy on those high-impact options.

#### **Example Use Case:**
Deciding which tasks to prioritize at work to maximize productivity.

---

### **4. The Pros and Cons List**
#### **Overview:**
This classic framework is simple and effective for weighing the advantages and disadvantages of a decision. It’s useful for straightforward choices.

#### **How It Works:**
- Divide a sheet of paper into two columns: "Pros" and "Cons."
- List all the positive outcomes (Pros) and negative outcomes (Cons) of the decision.
- Compare the two lists to make an informed choice.

#### **Example Use Case:**
Deciding whether to accept a new job offer or stay in your current role.

---

### **5. The OODA Loop (Observe, Orient, Decide, Act)**
#### **Overview:**
Originally developed for military strategy, the OODA Loop is a dynamic framework for making decisions in rapidly changing situations. It’s great for iterative decision-making.

#### **How It Works:**
- **Observe:** Gather information about the situation.
- **Orient:** Analyze the information and understand your options.
- **Decide:** Choose the best course of action.
- **Act:** Implement the decision, then repeat the loop as necessary.

#### **Example Use Case:**
Deciding how to respond to a sudden change in plans, like a canceled meeting or unexpected opportunity.

---

### **Comparison and Application:**
| Framework         | Best For                              | Complexity | Time Required |
|--------------------|---------------------------------------|------------|---------------|
| Eisenhower Matrix  | Prioritizing tasks                   | Low        | Quick         |
| 10/10/10 Rule      | Evaluating short vs. long-term impact| Low        | Quick         |
| Pareto Principle   | Optimizing effort for results        | Low        | Moderate      |
| Pros and Cons List | Simple trade-off decisions           | Low        | Quick         |
| OODA Loop          | Dynamic, iterative decisions         | Moderate   | Moderate      |

---

### **Recommendation:**
- **For daily prioritization:** Use the Eisenhower Matrix.
- **For long-term perspective:** Apply the 10/10/10 Rule.
- **For efficiency:** Leverage the Pareto Principle.
- **For straightforward choices:** Create a Pros and Cons List.
- **For adaptive situations:** Use the OODA Loop.

These frameworks can be combined or adapted based on the complexity and nature of your decision-making needs.

### Step 7 - Execute a Decision Scenario with a System Prompt (Custom AI Persona)

In this step a decision scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full **Decision Intelligence** framework will not be used, rather a simple request for Artificial Intelligence to recommend a path forward (recommendation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are Alex's main decision factors: financial, career uncertainty, academic consistency and future impact. In addition, you have all the decision factor detailed data available to pass to the GenAI model prompt. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence be that impartial judge to help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [11]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed Decision Scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommendation in a Markdown table format. 

Financial Considerations:
Alex's four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor's degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to build early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. 

Future Impact: 
Alex is unsure of the short-term future impact of her decision that might be hard to remediate. 
Alex wants a solid professional network and relevant experience when she graduates. 
Alex is not overly concerned about the social aspect of college, 
but feels she can build a quality network in a four-year university setting sooner. 
She is also concerned about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. 
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{scenarioDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, // Increase the max tokens to allow for a more detailed response
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseScenario = await semanticKernel.InvokePromptAsync(scenarioDecisionPromptTemplate, kernelArguments);
var response = promptResponseScenario.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

### Recommendation Table: Should Alex Enroll Directly in a Four-Year University or Start at a Community College?

| **Decision Factor**            | **Four-Year University**                                                                 | **Community College**                                                                 | **Recommendation**                                                                 |
|--------------------------------|----------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------|
| **Financial Considerations**   | High cost ($50,000-$60,000/year); family can afford with loans, but debt is a concern.  | Low cost ($3,000-$5,000/year); substantial savings; transferring may add costs later.| Community college offers significant savings, reducing financial stress and long-term debt. |
| **Career and Major Uncertainty** | Risk of switching majors could extend time and cost; limited flexibility to explore.    | Greater flexibility to explore interests without significant financial consequences. | Community college allows Alex to explore majors with lower financial risk.        |
| **Academic Consistency and Networking** | Early access to professors, internships, and campus groups; stronger continuity.     | Delayed access to networking opportunities; adjusting to a new campus later.         | Four-year university offers earlier access to networking, but community college provides time to focus. |
| **Future Impact**              | Builds professional network sooner; potential for higher debt impacting long-term finances. | Lower debt improves financial stability; transferring may require adjustment but is manageable. | Community college minimizes financial risk while still allowing later networking. |

### Final Recommendation:
Based on the decision factors, **Alex should start at a community college**. This option provides the flexibility to explore her interests at a lower financial cost, reducing the risk of incurring significant student loan debt if she changes her major. While transferring to a four-year university later may require adjustment, Alex can still build a professional network and gain relevant experience during her final two years. This approach balances her academic uncertainty, financial concerns, and long-term career prospects effectively.